# Amazon Bedrock AgentCore Runtime에서 Amazon Bedrock model을 사용하는 Strands Agent 호스팅

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore Runtime을 사용하여 기존 agent를 호스팅하는 방법을 배웁니다. Amazon Bedrock model과 Azure OpenAI 및 Gemini와 같은 non-Bedrock model을 사용하는 예제를 제공합니다.


### 튜토리얼 세부 정보


| 정보         | 세부 사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 대화형                                                                   |
| Agent 유형          | 단일                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                        |
| 튜토리얼 구성 요소 | AgentCore Runtime에서 agent 호스팅. Strands Agent 및 Amazon Bedrock Model 사용 |
| 튜토리얼 분야   | 범용                                                                   |
| 예제 복잡도  | 쉬움                                                                             |
| 사용된 SDK            | Amazon BedrockAgentCore Python SDK 및 boto3                                     |

### 튜토리얼 아키텍처

이 튜토리얼에서는 기존 agent를 AgentCore runtime에 배포하는 방법을 설명합니다. 

데모 목적으로 Amazon Bedrock model을 사용하는 Strands Agent를 사용합니다.

예제에서는 두 가지 tool인 `get_weather`와 `get_time`을 가진 매우 간단한 agent를 사용합니다. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### 튜토리얼 주요 기능

* Amazon Bedrock AgentCore Runtime에서 Agent 호스팅
* Amazon Bedrock model 사용
* Strands Agent 사용


## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands Agents

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## agent 생성 및 로컬 실험

agent를 AgentCore Runtime에 배포하기 전에 실험 목적으로 로컬에서 개발하고 실행해 보겠습니다.

프로덕션 agentic 애플리케이션의 경우 agent 생성 프로세스와 agent 호출 프로세스를 분리해야 합니다. AgentCore Runtime을 사용하면 agent의 호출 부분을 `@app.entrypoint` decorator로 장식하고 이를 runtime의 진입점으로 사용합니다. 먼저 실험 단계에서 각 agent가 어떻게 개발되는지 살펴보겠습니다.

여기서 아키텍처는 다음과 같습니다:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [ ]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # calculator tool import
import argparse
import json
from strands.models import BedrockModel

# 커스텀 tool 생성
@tool
def weather():
    """ Get weather """ # 더미 구현
    return "sunny"


model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
# Agent 초기화: model, tools, system_prompt 설정
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    # response.message에서 첫 번째 content의 text 추출
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    # JSON 문자열을 dict로 변환하여 agent 호출
    response = strands_agent_bedrock(json.loads(args.payload))

#### 로컬 agent 호출

In [ ]:
!python strands_claude.py '{"prompt": "What is the weather now?"}'

## AgentCore Runtime에 배포하기 위한 agent 준비

이제 agent를 AgentCore Runtime에 배포해 보겠습니다. 이를 위해 다음을 수행해야 합니다:
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp`으로 Runtime App 가져오기
* 코드에서 `app = BedrockAgentCoreApp()`으로 App 초기화
* `@app.entrypoint` decorator로 호출 함수 장식
* `app.run()`으로 AgentCoreRuntime이 agent 실행을 제어하도록 설정

### Amazon Bedrock model을 사용하는 Strands Agent
Amazon Bedrock model을 사용하는 Strands Agent부터 시작하겠습니다. 다른 모든 agent도 정확히 동일하게 작동합니다.

In [ ]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # calculator tool import
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

# AgentCore Runtime App 초기화
app = BedrockAgentCoreApp()

# 커스텀 tool 생성
@tool
def weather():
    """ Get weather """ # 더미 구현
    return "sunny"


model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
# Agent 초기화: model, tools, system_prompt 설정
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

# AgentCore Runtime의 진입점으로 등록
@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    # response.message에서 첫 번째 content의 text 추출
    return response.message['content'][0]['text']

if __name__ == "__main__":
    # AgentCore Runtime이 agent 실행을 제어하도록 설정
    app.run()

## 내부에서 무슨 일이 일어나나요?

`BedrockAgentCoreApp`을 사용하면 자동으로:

* 포트 8080에서 수신 대기하는 HTTP 서버 생성
* agent의 요구 사항을 처리하기 위한 필수 `/invocations` 엔드포인트 구현
* 상태 확인을 위한 `/ping` 엔드포인트 구현 (비동기 agent에 매우 중요)
* 적절한 콘텐츠 유형 및 응답 형식 처리
* AWS 표준에 따른 오류 처리 관리

## AgentCore Runtime에 agent 배포

`CreateAgentRuntime` 작업은 포괄적인 구성 옵션을 지원하여 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있습니다. 또한 프로토콜 설정(HTTP, MCP) 및 권한 부여 메커니즘을 구성하여 클라이언트가 agent와 통신하는 방법을 제어할 수 있습니다. 

**참고:** 운영 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인 및 IaC를 사용하여 ECR에 푸시하는 것입니다.

이 튜토리얼에서는 Amazon Bedrock AgentCore Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore runtime에 배포할 수 있습니다.

### AgentCore Runtime 배포 구성

먼저 스타터 툴킷을 사용하여 진입점, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore Runtime 배포를 구성합니다. 또한 시작 시 Amazon ECR 리포지토리를 자동으로 생성하도록 스타터 킷을 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 docker 파일이 생성됩니다.

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_claude_getting_started"
# AgentCore Runtime 배포 구성: 진입점, 실행 역할, ECR, requirements 설정
response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,  # 실행 역할 자동 생성
    auto_create_ecr=True,  # ECR 리포지토리 자동 생성
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

### AgentCore Runtime에 agent 시작

이제 docker 파일을 얻었으므로 agent를 AgentCore Runtime에 시작해 보겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore Runtime이 생성됩니다.

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore Runtime 상태 확인
이제 AgentCore Runtime을 배포했으므로 배포 상태를 확인해 보겠습니다.

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
# 최종 상태가 될 때까지 10초마다 상태 확인
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtime 호출

마지막으로 페이로드를 사용하여 AgentCore Runtime을 호출할 수 있습니다.

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

### 호출 결과 처리

이제 호출 결과를 처리하여 애플리케이션에 포함할 수 있습니다.

In [ ]:
from IPython.display import Markdown, display
import json
# 응답에서 첫 번째 텍스트 추출
response_text = invoke_response['response'][0]
display(Markdown(response_text))

### boto3로 AgentCore Runtime 호출

이제 AgentCore Runtime이 생성되었으므로 모든 AWS SDK로 호출할 수 있습니다. 예를 들어 boto3 `invoke_agent_runtime` 메서드를 사용할 수 있습니다.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# boto3로 AgentCore Runtime 호출
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"})
)
# 응답 타입이 event-stream인 경우 처리
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    # 일반 응답 처리
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## 정리 (선택 사항)

이제 생성된 AgentCore Runtime을 정리해 보겠습니다.

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

# AgentCore Runtime 삭제
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

# ECR 리포지토리 삭제 (force=True로 이미지 포함 삭제)
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# 축하합니다!